In [1]:
import xarray as xr
import bgcalc_xr as bgcalc

In [2]:
def set_model(model_path):
    ds = xr.open_dataset(model_path)
    hmin = 12.
    hmax = 50.

    lonl = -95
    lonr = -91
    latl = 28.
    latu = 29.6

    mask = (ds.h >= hmin) & (ds.h <= hmax) & \
        (ds.coords['lon_rho'] >= lonl) & (ds.coords['lon_rho'] <= lonr) & \
        (ds.coords['lat_rho'] >= latl) & (ds['lat_rho'] < latu) & \
        (ds.mask_rho == 1)
    mask[161:166, 274:279] = False
    ds.coords['mask2D'] = (('eta_rho', 'xi_rho'), mask)
    ds = ds.sel(ocean_time=slice("2010-08-01T00:00:00", "2010-09-30T23:30:00"))
    return bgcalc.Calculator(ds)


def extract(tidx):
    budget.subset(tidx)
    i_mx = budget.integ_matrix()
    dV = budget.dA * budget.dz_rho * i_mx
    print('loading')
    rate = diag['dye_01_rate'].isel(ocean_time=tidx).where(budget.ROI)
    hadv = diag['dye_01_hadv'].isel(ocean_time=tidx).where(budget.ROI)
    vadv = diag['dye_01_vadv'].isel(ocean_time=tidx).where(budget.ROI)
    vdiff = diag['dye_01_vdiff'].isel(ocean_time=tidx).where(budget.ROI)

    rate_vol = (rate * dV).sum(dim='s_rho')
    hadv_vol = (hadv * dV).sum(dim='s_rho')
    vadv_vol = (vadv * dV).sum(dim='s_rho')
    vdiff_vol = (vdiff * dV).sum(dim='s_rho')
    return dV, rate_vol, hadv_vol, vadv_vol, vdiff_vol


def run_file(budget, diag):
    steps = len(diag.ocean_time)
    print('starting iterations')
    for tidx in range(steps):
        print(tidx)
        try:
            dV, rate, hadv, vadv, vdiff = extract(tidx)
            if tidx == 0:
                print('initializing tsteps: ' + str(tidx))
                rate_vol = rate
                hadv_vol = hadv
                vadv_vol = vadv
                vdiff_vol = vdiff
            else:
                print('filling:' + str(tidx))
                print('rate')
                rate_vol = xr.concat([rate_vol, (rate * dV).sum(axis=0)], 'ocean_time')
                print('hadv')
                hadv_vol = xr.concat([hadv_vol, (hadv * dV).sum(axis=0)], 'ocean_time')
                print('vadv')
                vadv_vol = xr.concat([vadv_vol, (vadv * dV).sum(axis=0)], 'ocean_time')
                print('vdiff')
                vdiff_vol = xr.concat([vdiff_vol, (vdiff * dV).sum(axis=0)], 'ocean_time')
            print('filled tstep ' + str(tidx))
        except BaseException:
            print('conection droped at ' + str(tidx))
    new_ds = xr.Dataset({'rate': rate_vol, 'hadv': hadv_vol, 'vadv': vadv_vol, 'vdiff': vdiff_vol})
    new_ds.to_netcdf('diag_budget_08_2010.nc')

In [3]:
budget = set_model('http://copano.tamu.edu:8080/thredds/dodsC/NcML/txla_hindcast_agg')
diag = xr.open_mfdataset('/copano/d1/shared/tmp/TXLA_ROMS/2010_diag/ocean_dia*.nc')

/Users/vrx/miniconda3/envs/owl/lib/python3.6/_collections_abc.py:720: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  yield from self._mapping


OSError: no files to open